In [11]:
from astropy.modeling import core
from gwcs import WCS, utils
from jwst import datamodels
from jwst.assign_wcs import AssignWcsStep, nirspec as nrs, pointing
from jwst.assign_wcs import nirspec

### Create a WCS object for a Nirspec fixed slits observation - 5 open slits

In [12]:
#fpath = '/internal/1/astropy/jwst/jw00023001001_01101_00001_NRS1_assign_wcs.fits'
fpath = '/Users/dencheva/jw00023001001_01101_00001_NRS1_assign_wcs.fits'

In [13]:
im = datamodels.open(fpath)
refs={}
step=AssignWcsStep() 
for f in step.reference_file_types:
    refs[f] = step.get_reference_file(im, f)

RefResolutionError: ERROR loading embedded ASDF: Can't handle 'tag:astropy.org:astropy/coordinates/frames/baseframe-1.0.0' as a file for mode 'r'

In [4]:
pipeline, bbox = nirspec.create_pipeline(im, refs, slit_y_range=(-.5, .5))

NameError: name 'im' is not defined

In [5]:
bbox

{0: ((-0.5, -354643297.0449991), (-0.5, -1063771957.8266004)),
 1: ((-0.5, -7765003.330154864), (-0.5, -25535904.213872854)),
 2: ((-0.5, -4095787.924444007), (-0.5, -14781486.131393682)),
 3: ((-0.5, -11890226.37843511), (-0.5, -47106375.593335696)),
 4: ((-0.5, 2047.5), (-0.5, 2047.5))}

In [6]:
wcsobj = WCS(pipeline, inputs=('x', 'y', 'slit'), selector=('slit',)) 
wcsobj.bounding_box = bbox

### Calling the WCS object, i.e. using set_inputs in the `WCS.__call__` method

In [7]:
print(wcsobj)

   From      Transform  
---------- -------------
  detector CompoundModel
       sca CompoundModel
       gwa CompoundModel
slit_frame CompoundModel
 msa_frame CompoundModel
     oteip CompoundModel
      v2v3 CompoundModel
     world          None


In [8]:
wcsobj.bounding_box

{0: ((-0.5, -354643297.0449991), (-0.5, -1063771957.8266004)),
 1: ((-0.5, -7765003.330154864), (-0.5, -25535904.213872854)),
 2: ((-0.5, -4095787.924444007), (-0.5, -14781486.131393682)),
 3: ((-0.5, -11890226.37843511), (-0.5, -47106375.593335696)),
 4: ((-0.5, 2047.5), (-0.5, 2047.5))}

In [9]:
wcsobj.inputs


('x', 'y', 'slit')

In [10]:
wcsobj.selector

('slit',)

In [42]:
forward_transform = wcsobj.forward_transform
transform = core.set_inputs(forward_transform, {2: 0})

bb = wcsobj.bounding_box[list(fixed_inputs.keys())[0]]
transform.bounding_box = bb

In [43]:
# Various checks done meanwhile:
for key in fixed_inputs.keys():
    if utils.isnumerical(key) and key > len(wcsobj.inputs) or wcsobj.inputs.index(wcsobj.selector[0]) != key:
        raise ValueError("")
    elif isinstance(key, str) and key not in wcsobj.selector:
        raise ValueError("The inputs that can be fixed are {}".format(self.selector))

In [44]:
# To evaluate the transform I need to remove the fixed argument in WCS.__call__
# assuming integer key
args = [1000, 1074, 0]
key = list(fixed_inputs.keys())[0]
args.remove(args[key])


In [45]:
# Now I can call the new transform
transform(*args)


(15.154044390141793, 15.09279127041945, 1.0525618560405021, 0.0)

The user facing interface is in this case (i.e. calling the WCS ):

In [41]:
kwargs={'fixed_inputs':{2:0}} 
# wcsobj(1, 2, 0, **kwargs) # this is how it will be called

Ideally I would prefer to call it liek

```
wcsobj(1, 2, slit=0)
```

In [16]:
# The reason for the clumsiness is that the bounding_box is dealt with in the WCS object

# Create a new unique  WCS using set_inputs

Ideally I'd like to be able to go through the pipeline steps and for each step extract the unique transform using set_inputs and construct a new pipeline. This means however that either the fixed input needs to be in the same positional place for all transforms or it needs to use the same name. For example, for a fixed input, called `oder` which is in third position in the inputs to the WCS and a pipeline like

```
[(detector, transform1),
 (focal, transform2),
 (world, None)
 ]
```

- `transform1` and `transform2` need to have inut named `order`
- input `order` needs to be in the same position in the inputs for both transforms.

If this was possible the following would work:

In [17]:
def set_inputs(wcs, **fixed):
    new_pipeline = []
    for step in self.pipeline:
        transform = core.set_inputs(step[1], fixed)
        new_pipeline.append((step[0], transform))
    
    return self.__class__(new_pipeline)

Currently modeling assigns generic names to the inputs and it's not possible to change them. For a good reason - `Model.inputs` defines `Model.n_inputs`. There's also code which matches inptus to outputs in compound models to facilitate working with units.

Assuming the fixed input would be in the same position for all transforms is not good either. Often the models require 
an intermediate `Mapping` to change the order of inputs.

In [21]:
set_inputs(wcsobj, slit=0)

ValueError: Substitution key string not among possible input choices

So currently it's only possible to return a complete WCS with one transform only, combining the individual pipeline steps into a single step.

In [32]:
def set_inputs1(wcs, **fixed):
    new_pipeline = []
    new_pipeline.append((wcs.input_frame,
                         core.set_inputs(wcs.forward_transform, fixed)))
    new_pipeline.append((wcs.output_frame, None))
    wcsobj = WCS(new_pipeline)
    wcsobj.bounding_box = wcs._compound_bbox[list(fixed.values())[0]]
    return wcsobj


The last line in the function is a bit misleading. It works if there's only one fixed input.
Let's say we have several inputs and a combination is required to define a unique WCS - `order`, `x_source`, `y_source`.

I imagine the bounding box in this case will look like

```
wcsobj._compound_bbox

{(1, 1089, 1000): [[-0.5, -354643297.0449991], [-0.5, -1063771957.8266004]],
 (2, 100, 200): [[-0.5, -7765003.330154864], [-0.5, -25535904.213872854]]
}
```

where the keys are tuples of `(order, x_source, y_source)`.

The WCS should have an attribute which tells users which inputs can be used as a selector.
For example, `wcsobj.selector = ("order", "x_source", "y_source")`. 

So the function above needs to become:
```
def set_inputs(wcs, **fixed):
    new_pipeline = []
    new_pipeline.append((wcs.input_frame,
                         core.set_inputs(wcs.forward_transform, fixed)))
    new_pipeline.append((wcs.output_frame, None))
    wcsobj = WCS(new_pipeline)
    keys = [fixed[key] for key in wcsobj.selector)
    wcsobj.bounding_box = wcs._compound_bbox[keys]
    
```

Does this make sense?


Is it possible to make it work with positional arguments? We need to explore allowing `Model.inputs` to be settable although some of the basic Model functionality depends on this attribute.

In [33]:
new_wcs = set_inputs1(wcsobj, x01=0)

In [35]:
new_wcs(1000, 1070)

fixed {}


[nan, nan, nan, nan]

In [37]:
new_wcs.bounding_box # bounding box is currently computed incorrectly

((-0.5, -354643297.0449991), (-0.5, -1063771957.8266004))

In [38]:
new_wcs(1000, 1070, with_bounding_box=False)

fixed {}


(15.153991035614474, 15.092896384943735, 1.0526474535394414, 0.0)